In [1]:
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import os
import datetime
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

import pickle
import sys
import matplotlib.pyplot as plt
# just set the seed for the random number generator
np.random.seed(107)

import numpy as np
np.random.seed(1) # NumPy
import random
random.seed(3) # Python
import tensorflow as tf

from keras import backend as K

import matplotlib.cm as cm

from sklearn.manifold import TSNE

from keras.layers import Dense, Flatten, LSTM, Dropout

In [2]:
# Usando Google Colab:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/Python/MEU_TCC/Classes')
pasta = str("/content/drive/MyDrive/Python/MEU_TCC/Data/")
print(pasta)

preços = pd.read_csv(pasta + "Preços.csv",index_col='Data',parse_dates=True)
semestres = pd.read_csv(pasta + "Semestres.csv",parse_dates=True)

Datas = pd.DataFrame(preços.index)
Datas['indice'] = Datas.index
log_data = np.log(preços)


Mounted at /content/drive
/content/drive/MyDrive/Python/MEU_TCC/Data/


In [3]:
%load_ext autoreload
%aimport class_SeriesAnalyser_Distance, class_Trader_Distance, class_DataProcessor
%autoreload 1

series_analyser = class_SeriesAnalyser_Distance.SeriesAnalyser()
trader = class_Trader_Distance.Trader()
data_processor = class_DataProcessor.DataProcessor()

In [4]:
top_n = 20
subsample = 2500
min_half_life = 0 # number of points in a day
max_half_life = 200000 #~number of points in a year: 78*252
file_extension = 'intraday'

# Calculando Renda fixa anualizada:

In [ ]:
pastaC = str("/content/drive/MyDrive/Python/MEU_TCC/Data/")
CDI_ret = pd.read_csv(pastaC + "CDI_ret.csv", )
# Convert the 'data' column to datetime format
CDI_ret.index = pd.to_datetime(CDI_ret['Data'])
CDI_ret.drop(['Data'],inplace=True,axis=1)

CDI_ret = CDI_ret[1:]

In [ ]:
anoi = 1996
rfannualized = pd.DataFrame(columns=['retorno_anualizado'])
inicio = 0

for nsem in range(0, len(semestres) - 2,2):
    print(f"Starting period {nsem} | Past days: {inicio}")
    ano = math.floor(anoi + nsem/2)
    print(ano)

    # Listando os dias dos intervalos
    inicio = (Datas == semestres['Data'][nsem]).query("Data == True").index[0]
    inicio = Datas.iloc[inicio][0]

    twelve_months = (Datas == semestres['Data'][nsem + 2]).query("Data == True").index[0]
    twelve_months = Datas.iloc[twelve_months][0]
    print('inicio: ',inicio,' twelve_months: ',twelve_months)


    annualized_ret = (np.cumprod(1 + CDI_ret['CDI Acumulado'][inicio:twelve_months]) - 1)[-1]
    rfannualized.loc[ano] = annualized_ret

rf_mean_ano = rfannualized.mean()[0]

Starting period 0 | Past days: 0
1996
inicio:  1996-01-02 00:00:00  twelve_months:  1996-12-31 00:00:00
Starting period 2 | Past days: 1996-01-02 00:00:00
1997
inicio:  1996-12-31 00:00:00  twelve_months:  1997-12-31 00:00:00
Starting period 4 | Past days: 1996-12-31 00:00:00
1998
inicio:  1997-12-31 00:00:00  twelve_months:  1998-12-31 00:00:00
Starting period 6 | Past days: 1997-12-31 00:00:00
1999
inicio:  1998-12-31 00:00:00  twelve_months:  1999-12-31 00:00:00
Starting period 8 | Past days: 1998-12-31 00:00:00
2000
inicio:  1999-12-31 00:00:00  twelve_months:  2000-12-29 00:00:00
Starting period 10 | Past days: 1999-12-31 00:00:00
2001
inicio:  2000-12-29 00:00:00  twelve_months:  2001-12-31 00:00:00
Starting period 12 | Past days: 2000-12-29 00:00:00
2002
inicio:  2001-12-31 00:00:00  twelve_months:  2002-12-31 00:00:00
Starting period 14 | Past days: 2001-12-31 00:00:00
2003
inicio:  2002-12-31 00:00:00  twelve_months:  2003-12-31 00:00:00
Starting period 16 | Past days: 2002-12

#No Cluster - Distância

In [ ]:
inicio = 0
pairs_t = []
performance_threshold_test_t = []
sharpe_results_threshold_test_nocosts_t = []
cum_returns_threshold_test_nocosts_t = []
results_t = pd.DataFrame(columns=['n_pairs','Daily_sharpe_ratio','avg_annual_roi',
                                  'Total Number of Trades','Positive Trades','Negative Trades','pct_positive_trades_per_pair',
                                  'pct_pairs_with_positive_returns','avg_half_life','avg_hurst_exponent','Maximum Drawdown'])

Pares_Total = pd.DataFrame(columns=['Ticker 1','Ticker 2','Erro Quadrático Médio'])

for big_loop in range(0, len(semestres) - 3):
    print(f"\nStarting period {big_loop} | Past days: {inicio}")

    # Listando os dias dos intervalos
    inicio = (Datas == semestres['Data'][big_loop]).query("Data == True").index[0]
    inicio = Datas.iloc[inicio][0]
    twelve_month = (Datas == semestres['Data'][big_loop + 2]).query("Data == True").index[0]
    twelve_months = Datas.iloc[twelve_month][0]
    inicio_test = Datas.iloc[twelve_month+1][0]
    six_months = (Datas == semestres['Data'][big_loop + 3]).query("Data == True").index[0]
    six_months = Datas.iloc[six_months][0]

    # split data in training and test
    df_prices_train, df_prices_test = data_processor.split_data(preços,(inicio, twelve_months),(inicio_test,six_months),remove_nan=True)

    # interday
    n_years_test = round(len(df_prices_train)/(250))

    no_cluster_series = pd.Series(0,index = df_prices_train.columns)
    pairs_all_a_all, unique_tickers = series_analyser.get_candidate_pairs(clustered_series=no_cluster_series,
                                                                          pricing_df_train=df_prices_train,
                                                                          pricing_df_test=df_prices_test,
                                                                          min_half_life=min_half_life,
                                                                          max_half_life=max_half_life,
                                                                          min_zero_crosings=12,
                                                                          hurst_threshold= 1,
                                                                          subsample=0,
                                                                          top_n=top_n
                                                                          )
    print('Out of {} possible combinations'.format(len(df_prices_train.columns)*(len(df_prices_train.columns)-1)/2))

    results_without_costs, results_with_costs, performance_threshold_test = \
            trader.apply_trading_strategy(pairs_all_a_all,
                                          strategy='fixed_beta',
                                          entry_multiplier=2,
                                          exit_multiplier=0,
                                          test_mode=True,
                                          train_val_split=0,
                                          rf = rfannualized
                                          )

    sharpe_results_threshold_test_nocosts, cum_returns_threshold_test_nocosts = results_without_costs
    #sharpe_results_threshold_test_w_costs, cum_returns_threshold_test_w_costs = results_with_costs

    # interday
    n_years_test = round(len(df_prices_test)/(240))

    n_years_test

    results, pairs_summary = trader.summarize_results(sharpe_results_threshold_test_nocosts,
                                                      cum_returns_threshold_test_nocosts,
                                                      performance_threshold_test,
                                                      pairs_all_a_all,
                                                      n_years_test,
                                                      rf = rfannualized)

    for i in range(0, len(pairs_all_a_all)):
      Pares_Total = Pares_Total.append({'Ticker 1': pairs_all_a_all[i][0], 'Ticker 2': pairs_all_a_all[i][1], 'Erro Quadrático Médio': pairs_all_a_all[i][2]['SSE']},ignore_index=True)

    results_t = results_t.append(results,ignore_index=True)

results_t['Annualized SR'] = results_t['Daily_sharpe_ratio']*(252**0.5)

Pares_Total['Ticker Combinado'] = Pares_Total[['Ticker 1', 'Ticker 2']].apply(lambda x: '-'.join(sorted(x)), axis=1)
Pares_Total = Pares_Total.drop_duplicates(subset=['Ticker Combinado'], keep='first')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
pd.set_option('display.float_format', '{:.10e}'.format)


In [ ]:
pastaP = str("/content/drive/MyDrive/Python/MEU_TCC/Data/pickle/")

with open(pastaP + 'Pares_Distancia_Fixed_Beta_20.pickle', 'wb') as f:
    pickle.dump(results_t, f)

with open(pastaP + 'Pares_Distancia_Lista_20.pickle', 'wb') as f:
    pickle.dump(Pares_Total, f)

In [5]:
pastaP = str("/content/drive/MyDrive/Python/MEU_TCC/Data/pickle/")

with open(pastaP + 'Pares_Distancia_Fixed_Beta_20.pickle', 'rb') as handle:
    results_t = pickle.load(handle)

with open(pastaP + 'Pares_Distancia_Lista_20.pickle', 'rb') as handle:
    Pares_Total = pickle.load(handle)

In [6]:
results_t

,n_pairs,Daily_sharpe_ratio,avg_annual_roi,Total Number of Trades,Positive Trades,Negative Trades,pct_positive_trades_per_pair,pct_pairs_with_positive_returns,avg_half_life,avg_hurst_exponent,Maximum Drawdown,Annualized SR
0,20,0.028355,16.517139,25,12,13,40.833333,45.0,33.75,0.344262,-5.249227,0.450121
1,20,0.067270,24.123125,29,21,8,68.333333,75.0,47.75,0.249715,-5.691801,1.067875
2,20,-0.045750,6.227313,25,15,10,53.333333,55.0,20.45,0.301062,-8.927472,-0.726256
3,20,-0.057007,5.282568,22,13,9,55.000000,55.0,15.25,0.201328,-7.574565,-0.904954
4,20,0.042819,30.133657,21,9,12,42.500000,45.0,28.60,0.270814,-10.173339,0.679728
5,20,0.060676,22.081410,24,17,7,65.000000,65.0,33.55,0.246878,-4.515722,0.963209
6,20,0.197910,40.629717,24,20,4,82.500000,85.0,76.20,0.326549,-2.778584,3.141721
7,20,-0.094796,-17.052725,20,6,14,30.000000,30.0,26.30,0.364769,-23.323711,-1.504835
8,20,-0.050913,2.331899,23,12,11,47.500000,50.0,20.95,0.311163,-8.159523,-0.808213
9,20,-0.002416,-13.290871,20,7,13,35.000000,35.0,24.55,0.234370,-26.262218,-0.038359


In [ ]:
#Sharpe Ratio Anualizado:
SRA = (results_t['avg_annual_roi'].mean())/np.std(results_t['avg_annual_roi'])

In [ ]:
# Resultados:
print('Top N Pares:',results_t['n_pairs'].min())
#print('Sharpe Ratio Anualizado: {:.3f}'.format(results_t['Annualized SR'].mean()))
print('Sharpe Ratio Anualizado: {:.3f}'.format(SRA))
print('Excesso de Retornos Anualizado: {:.3f} %'.format(results_t['avg_annual_roi'].mean()))
print('MDD: {:.3f} %'.format(results_t['Maximum Drawdown'].min()))
print('Total de Pares:', Pares_Total['Ticker Combinado'].count()) # Não está contando a repetição dos pares
print('Pares rentáveis: {:.3f} %'.format(results_t['pct_pairs_with_positive_returns'].mean()))
print('Total de Transações:', results_t['Total Number of Trades'].sum())
print('Operações rentáveis:', results_t['Positive Trades'].sum())
print('Operações deficitárias :', results_t['Negative Trades'].sum())

Top N Pares: 20
Sharpe Ratio Anualizado: 0.314
Excesso de Retornos Anualizado: 3.521 %
MDD: -26.262 %
Total de Pares: 516
Pares rentáveis: 50.926 %
Total de Transações: 1205
Operações rentáveis: 637
Operações deficitárias : 568


In [ ]:
rf_mean_ano*100

14.244056961092486

In [ ]:
results_t['avg_annual_roi'] + rf_mean_ano*100

0     30.761196
1     38.367182
2     20.471370
3     19.526625
4     44.377714
5     36.325467
6     54.873774
7     -2.808668
8     16.575956
9      0.953186
10    -5.577321
11    33.654355
12     5.602897
13    20.123252
14    16.799278
15    11.199330
16     9.225027
17    19.731456
18    25.306596
19    18.489412
20    21.880771
21    16.891840
22    24.310833
23    20.507671
24    16.080659
25     7.086336
26    10.880650
27    21.266857
28    10.043642
29    16.755478
30    13.667472
31    15.324627
32    11.516514
33    19.869290
34    16.676793
35    17.047284
36    12.089330
37     6.026209
38     5.981996
39    18.382593
40    26.459436
41     7.889170
42     7.297448
43     1.156777
44     8.346400
45    20.829718
46    33.192123
47    23.857037
48    13.938348
49    14.112116
50    15.733269
51    19.576103
52    34.099759
53    16.572186
Name: avg_annual_roi, dtype: float64

In [ ]:
rfsem = []
rfsem[rfannualized.iloc[0][0]]
for s in range(1,len(rfannualized)):
  print(rfannualized.iloc[s])
  rfsem = rfsem.append(rfannualized.iloc[s][0],ignore_index=True)

TypeError: list indices must be integers or slices, not numpy.float64

In [ ]:
rfsem[rfannualized.iloc[0][0]]
rfsem

TypeError: list indices must be integers or slices, not numpy.float64

In [ ]:
results_t['Annualized SR'].mean()

-0.5695879871781256

In [ ]:
results_t

,n_pairs,Daily_sharpe_ratio,avg_annual_roi,Total Number of Trades,Positive Trades,Negative Trades,pct_positive_trades_per_pair,pct_pairs_with_positive_returns,avg_half_life,avg_hurst_exponent,Maximum Drawdown,Annualized SR
0,20,0.028355,16.517139,25,12,13,40.833333,45.0,33.75,0.344262,-5.249227,0.450121
1,20,0.067270,24.123125,29,21,8,68.333333,75.0,47.75,0.249715,-5.691801,1.067875
2,20,-0.045750,6.227313,25,15,10,53.333333,55.0,20.45,0.301062,-8.927472,-0.726256
3,20,-0.057007,5.282568,22,13,9,55.000000,55.0,15.25,0.201328,-7.574565,-0.904954
4,20,0.042819,30.133657,21,9,12,42.500000,45.0,28.60,0.270814,-10.173339,0.679728
5,20,0.060676,22.081410,24,17,7,65.000000,65.0,33.55,0.246878,-4.515722,0.963209
6,20,0.197910,40.629717,24,20,4,82.500000,85.0,76.20,0.326549,-2.778584,3.141721
7,20,-0.094796,-17.052725,20,6,14,30.000000,30.0,26.30,0.364769,-23.323711,-1.504835
8,20,-0.050913,2.331899,23,12,11,47.500000,50.0,20.95,0.311163,-8.159523,-0.808213
9,20,-0.002416,-13.290871,20,7,13,35.000000,35.0,24.55,0.234370,-26.262218,-0.038359
